<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Semiveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lasting av nødvendige bibliotek og pakker

In [ ]:
!pip install LAMDA-SSL
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.0 MB/s eta 0:00:00


Importering av nødvendige bibliotek og pakker

In [ ]:
import pandas as pd
import numpy as np
import copy

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

from sklearn.metrics import mean_squared_error, r2_score

import optuna
from optuna.visualization import plot_optimization_history

Importering av relevant data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Velger første kolonne med dato og tid som index
collagen_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Formatterer index til riktig format og datatype
collagen_data.index = pd.to_datetime(collagen_data.index,
                                     format='%Y-%m-%d %H:%M:%S')

Mounted at /content/drive


Tilfeldighetsfrø

In [ ]:
#Tilfeldighetsfrø
random_seed = 123

# NIR målinger inkludert

Markert og umarkert del av datasettet

In [ ]:
# Deler opp markert del av datasettet
collagen_markert = collagen_data.dropna(subset='Collagen')
collagen_umarkert = collagen_data[collagen_data['Collagen'].isna()]

## Oppdeling av datasett til trening og testsett

### Alternativ 1: Fordelt etter Enzymtype

In [ ]:
# Legger til kategorisk variabel for enzymtype til fordeling av datsettet
collagen_markert['EnzymType'] = collagen_markert.filter(like='EnzymeType_').idxmax(axis=1).str.split('_').str[1].astype('category')

# Splitter datasett i collagendataasett og kategorisk enzymtype
collagen_enzymetypes = collagen_markert['EnzymType']
collagen_markert.drop(columns=['EnzymType'], inplace=True)

# Splitter trening og testdata etter enzymtype
collagen_trening_markert, collagen_test_markert, _, _ = train_test_split(collagen_markert,
                                                                         collagen_enzymetypes,
                                                                         test_size = 0.20,
                                                                         stratify = collagen_enzymetypes,
                                                                         random_state = random_seed)

# Deler opp datasettene til prediktorer og respons, for trening og testsett
X_trening_markert = collagen_trening_markert.iloc[:, :-1]
X_trening_umarkert = collagen_umarkert.iloc[:, :-1]
X_test = collagen_test_markert.iloc[:, :-1]
y_trening_markert = collagen_trening_markert.iloc[:, -1]
y_test = collagen_test_markert.iloc[:, -1]

# Lager et fullstendig treningssett, både markert og umarkert del av datasettet
collagen_prediktor_data = collagen_data.iloc[:, :-1]
felles_indeks = collagen_prediktor_data.index.intersection(X_test.index)
X_trening = collagen_prediktor_data.drop(felles_indeks)

<ipython-input-5-b7e95a4236ad>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collagen_markert['EnzymType'] = collagen_markert.filter(like='EnzymeType_').idxmax(axis=1).str.split('_').str[1].astype('category')
<ipython-input-5-b7e95a4236ad>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collagen_markert.drop(columns=['EnzymType'], inplace=True)


Skalerer data etter fullstendig treningsdata

In [ ]:
skalerer = StandardScaler()
skalerer.fit(X_trening)
X_trening_markert_transformert = skalerer.transform(X_trening_markert)
X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
X_test_transformert = skalerer.transform(X_test)

**CoReg**

Hyperparamter optimalisering med Optuna

In [ ]:
def objective(trial):
  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  return mse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=10)

plot_optimization_history(study)

[I 2024-02-09 10:29:21,189] A new study created in memory with name: no-name-0e6e59c1-9202-4135-8ac4-cfef5b9dfc85
[I 2024-02-09 10:31:33,774] Trial 0 finished with value: 13.787147646037315 and parameters: {'k1': 6, 'k2': 10, 'p1': 1, 'p2': 8}. Best is trial 0 with value: 13.787147646037315.
[I 2024-02-09 10:33:41,377] Trial 1 finished with value: 14.067948943850967 and parameters: {'k1': 6, 'k2': 2, 'p1': 5, 'p2': 6}. Best is trial 0 with value: 13.787147646037315.
[I 2024-02-09 10:36:03,144] Trial 2 finished with value: 15.294312736548319 and parameters: {'k1': 4, 'k2': 2, 'p1': 6, 'p2': 4}. Best is trial 0 with value: 13.787147646037315.
[I 2024-02-09 10:38:11,195] Trial 3 finished with value: 15.543994497955705 and parameters: {'k1': 10, 'k2': 3, 'p1': 3, 'p2': 7}. Best is trial 0 with value: 13.787147646037315.
[I 2024-02-09 10:40:25,258] Trial 4 finished with value: 14.655544421682329 and parameters: {'k1': 3, 'k2': 2, 'p1': 5, 'p2': 6}. Best is trial 0 with value: 13.78714764603

Evauleringer av beste modell

In [ ]:
def detailed_objective(trial):
  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)

  mse_trening = mean_squared_error(y_trening_markert, y_trening_prediksjon)
  r2_trening = r2_score(y_trening_markert, y_trening_prediksjon)

  return mse_test, r2_test, mse_trening, r2_trening

mse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mse_trening_resultat = detailed_objective(study.best_trial)[2]
r2_trening_resultat = detailed_objective(study.best_trial)[3]

print("Beste hyperparametere for CoReg: {}".format(study.best_params))
print("Test: MSE: {}, og R^2: {}".format(mse_test_resultat, r2_test_resultat))
print("Trening: MSE: {}, og R^2: {}".format(mse_trening_resultat, r2_trening_resultat))

Beste hyperparametere for CoReg: {'k1': 9, 'k2': 2, 'p1': 9, 'p2': 10}
Test: MSE: 14.62126212740936, og R^2: 0.5750354914795336
Trening: MSE: 1.6441313422956731, og R^2: 0.9214526096644635


### Alternativ 2: Fordelt på ukentlig og daglig variasjon i produksjon m/kontinuitet

In [ ]:
# Lager nytt trening og testsett
collagen_trening2 = copy.deepcopy(collagen_markert)
collagen_test2 = pd.DataFrame(columns = list(collagen_trening2.columns))

# Definere andel av datasettet som testsett
andel = 0.2

# Går gjennom hver dag, hver uke og samler første andel av rader til testsett
for uke in collagen_markert.index.isocalendar().week.unique():
  for dag in collagen_markert.index.isocalendar().day.unique():
    data = collagen_markert[(collagen_markert.index.isocalendar().week == uke) &
     (collagen_markert.index.isocalendar().day == dag)]
    test_rader = (data.shape[0])*andel
    test_data_dag = data.head(round(test_rader))

    collagen_test2 = pd.concat([collagen_test2, test_data_dag])
    collagen_trening2.drop(test_data_dag.index, inplace=True)

In [ ]:
# Deler opp datasettene til prediktorer og respons, for trening og testsett
X_trening_markert2 = collagen_trening2.iloc[:, :-1]
X_trening_umarkert = collagen_umarkert.iloc[:, :-1]
X_test2 = collagen_test2.iloc[:, :-1]
y_trening_markert2 = collagen_trening2.iloc[:, -1]
y_test2 = collagen_test2.iloc[:, -1]

# Lager et fullstendig treningssett, både markert og umarkert del av datasettet
collagen_prediktor_data = collagen_data.iloc[:, :-1]
felles_indeks = collagen_prediktor_data.index.intersection(X_test2.index)
X_trening2 = collagen_prediktor_data.drop(felles_indeks)

skalerer = StandardScaler()
skalerer.fit(X_trening2)
X_trening_markert_transformert2 = skalerer.transform(X_trening_markert2)
X_trening_umarkert_transformert2 = skalerer.transform(X_trening_umarkert)
X_test_transformert2 = skalerer.transform(X_test2)

**CoReg**

Hyperparamter optimalisering med Optuna

In [ ]:
def objective(trial):
  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  coreg.fit(X_trening_markert_transformert2, y_trening_markert2.values, X_trening_umarkert_transformert2)
  y_test_prediksjon = coreg.predict(X_test_transformert2)
  mse_test = mean_squared_error(y_test2, y_test_prediksjon)
  return mse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=10)

plot_optimization_history(study)

[I 2024-02-09 14:39:29,616] A new study created in memory with name: no-name-3f5ef366-942f-4c32-9e42-1ee9e08aeeac
[I 2024-02-09 14:41:51,464] Trial 0 finished with value: 12.308964014873418 and parameters: {'k1': 4, 'k2': 6, 'p1': 7, 'p2': 3}. Best is trial 0 with value: 12.308964014873418.
[I 2024-02-09 14:44:15,958] Trial 1 finished with value: 10.965744651806304 and parameters: {'k1': 6, 'k2': 4, 'p1': 10, 'p2': 9}. Best is trial 1 with value: 10.965744651806304.
[I 2024-02-09 14:46:41,403] Trial 2 finished with value: 10.522858866928095 and parameters: {'k1': 7, 'k2': 2, 'p1': 5, 'p2': 8}. Best is trial 2 with value: 10.522858866928095.
[I 2024-02-09 14:49:03,360] Trial 3 finished with value: 9.768933229134998 and parameters: {'k1': 7, 'k2': 9, 'p1': 5, 'p2': 8}. Best is trial 3 with value: 9.768933229134998.
[I 2024-02-09 14:51:30,047] Trial 4 finished with value: 13.296437269124802 and parameters: {'k1': 3, 'k2': 7, 'p1': 8, 'p2': 3}. Best is trial 3 with value: 9.768933229134998

Evauleringer av beste modell

In [ ]:
def detailed_objective(trial):
  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)

  mse_trening = mean_squared_error(y_trening_markert, y_trening_prediksjon)
  r2_trening = r2_score(y_trening_markert, y_trening_prediksjon)

  return mse_test, r2_test, mse_trening, r2_trening

mse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mse_trening_resultat = detailed_objective(study.best_trial)[2]
r2_trening_resultat = detailed_objective(study.best_trial)[3]

print("Beste hyperparametere for CoReg: {}".format(study.best_params))
print("Test: MSE: {}, og R^2: {}".format(mse_test_resultat, r2_test_resultat))
print("Trening: MSE: {}, og R^2: {}".format(mse_trening_resultat, r2_trening_resultat))

Beste hyperparametere for CoReg: {'k1': 9, 'k2': 2, 'p1': 9, 'p2': 10}
Test: MSE: 14.62126212740936, og R^2: 0.5750354914795336
Trening: MSE: 1.6441313422956731, og R^2: 0.9214526096644635
